# Pronóstico adaptativo

## Descripción del problema real

Los pronósticos de los precios de la electricidad en mercados liberalizados son un insumo fundamental para la toma de decisiones dentro de las organizaciones. Fundamentalmente, los pronosticos de corto plazo son utilizados en decisiones de carácter operativo. En el caso abordado, es necesario contar con los pronósticos para definir la politica de operación de una empresa del sector eléctrico.

## Descripción del problema en términos de los datos

La carpeta `datos/precios/` contiene los precios historicos horarios de la electricidad en la Bolsa de Energía del mercado eléctrico colombiano, publicados por el operador del mercado. Se desean obtener pronósticos para el precio promedio diario para los próximos siete (7) días a la fecha actual.

## Aproximaciones posibles

En este caso, se desea evaluar la capacidad de pronóstico de un ADALINE con aprendizaje en línea.

## Requerimientos

Usted debe:

* Procesar los datos históricos para conformar el conjunto de datos. Los archivos de Excel no pueden ser modificados y actualizados directamente por el operador del sistema. Su código debe leer los archivos y crear la serie de precios promedio diarios de la electricidad.


* Determinar si el modelo debe pronosticar los precios promedios sin ninguna transformación, o si la inclusión de una transformación (logaritmo natural, raíz cúbica, raíz cuadrada, etc) resulta en un incremento de la precisión del pronóstico.


* Generar el pronóstico para los precios de los próximos siete días.


* Preparar el código para que el modelo sea entrenado usando el gradiente y el momentum.


* Determinar cuál es el número óptimo de retardos (observaciones) que el modelo debe considerar hacia atrás para producir el pronóstico.


* Determinar los valores óptimos de la tasa de aprendizaje y el momentum.


# Angel Racini Meza
## C.C 1017248189


## Importanto Librerias

In [76]:
import pandas as pd
import numpy as np 
import os
from pathlib import Path
from matplotlib import pyplot as plt
from datetime import datetime
%matplotlib inline

## Preprocesando los datos

In [ ]:
#Extrayendo los archivos de la carpeta precios
dfs =  []
path  = r'C:\Users\ANGEL\Desktop\Trabajo Redes Neuronales\evaluacion-arracinim\datos\precios'
datos = os.listdir(path)

Consolidado = pd.read_excel(path+"\\"+datos[0])
Consolidado = Consolidado.iloc[:,0:25]
Consolidado = Consolidado.dropna()
Consolidado.rename(columns=Consolidado.iloc[0], inplace = True)
Consolidado = Consolidado.drop(Consolidado.index[0])
Consolidado.reset_index(drop=True, inplace=True)
Consolidado.columns = ['Fecha', "0", "1", "2", "3", "4", "5", "6", "7", "8", "9", "10", "11", "12", "13", "14",                             "15", "16", "17", "18", "19", "20", "21", "22", "23"]

file_number = 0

for archivo in datos:
    if file_number == 0:
        file_number = file_number + 1
    else:
        df = pd.read_excel(path+"\\"+archivo)
        df  = df.iloc[:,0:25]
        df = df.dropna()       
        df.rename(columns=Consolidado.iloc[0], inplace = True)
        df = df.drop(df.index[0])
        df.reset_index(drop=True, inplace=True)
        df.columns = ['Fecha', "0", "1", "2", "3", "4", "5", "6", "7", "8", "9", "10", "11", "12", "13", "14",                             "15", "16", "17", "18", "19", "20", "21", "22", "23"]
        Consolidado = pd.concat([Consolidado, df])

#IMPRIMIENDO LOS PRIMEROS 5 REGISTROS DEL ARCHIVO CONSOLIDADO
Consolidado.head(5)

In [150]:
#Obteniendo el promedio para cada dia
Precios_Promedio = Consolidado.set_index('Fecha')
Precios_Promedio['Precio_Promedio'] = Precios_Promedio.sum(axis = 1) / 24.0
Precios_Promedio = Precios_Promedio.loc[:,'Precio_Promedio']
Precios_Promedio = Precios_Promedio.reset_index()

#Leyendolos datos del banco de la republica
IPC = pd.read_excel(r"C:\Users\ANGEL\Desktop\Trabajo Redes Neuronales\evaluacion-arracinim\1.2.5.IPC_Serie_variaciones.xlsx", header = 12)
IPC = IPC.iloc[:,0:2] 
IPC = IPC.iloc[:-6] #No nos interesa las ultima 6 columnas correspondientes a los creditos
IPC.columns = ['Date_Month', 'IPC']

#Convierto las fechas a formato YYYY-MM

IPC['Date_Month'] = IPC['Date_Month'].astype(str)
IPC['Date_Month'] = IPC['Date_Month'].apply(lambda x: x[:4] + '-' + x[4:])
IPC['Date_Month'] = pd.to_datetime(IPC['Date_Month']).dt.to_period('M')
Precios_Promedio['Date_Month'] = pd.to_datetime(Precios_Promedio['Fecha']).dt.to_period('M')


#Haciendo un merge para calcular los nuevos precios
Precios = pd.merge(Precios_Promedio, IPC[["Date_Month", "IPC"]], on="Date_Month", how="left")

#Recalculando el valor del peso al valor actual segun el IPC para cada fila
IPC_ACTUAL = IPC.tail(1).iloc[0,1]
Precios['Precio_Promedio_Ajustado'] =  Precios['Precio_Promedio'] * (IPC_ACTUAL / Precios['IPC'])
Precios.head(5)

In [151]:
#Obteniendo las columnas que se requieren con el fin de obtener 
Precios = 

,Fecha,Precio_Promedio,Date_Month,IPC,Precio_Promedio_Ajustado
0,1995-07-21,4.924333,1995-07,20.93,24.689897
1,1995-07-22,1.269500,1995-07,20.93,6.365090
2,1995-07-23,0.953083,1995-07,20.93,4.778622
3,1995-07-24,4.305917,1995-07,20.93,21.589245
4,1995-07-25,1.149167,1995-07,20.93,5.761756
5,1995-07-26,1.108625,1995-07,20.93,5.558486
6,1995-07-27,0.585958,1995-07,20.93,2.937911
7,1995-07-28,0.499792,1995-07,20.93,2.505883
8,1995-07-29,0.927667,1995-07,20.93,4.651187
9,1995-07-30,0.669458,1995-07,20.93,3.356567
